In [17]:
import os
import cv2
import xml.etree.ElementTree as ET
from PIL import Image

In [18]:
import os
import cv2
import xml.etree.ElementTree as ET
from PIL import Image

def extract_text_from_image(min_page_number=0):

    input_image_folder = '../data/images/input_image/'
    input_xml_folder = '../data/images/input_xml/'
    output_image_folder = '../data/images/output/images/'
    output_label_folder = '../data/images/output/labels/'

    if not os.path.exists(output_image_folder):
        os.makedirs(output_image_folder)

    if not os.path.exists(output_label_folder):
        os.makedirs(output_label_folder)

    for xml_file in os.listdir(input_xml_folder):
        if xml_file.endswith('.xml'):
            tree = ET.parse(os.path.join(input_xml_folder, xml_file))
            root = tree.getroot()
            img_name = root.find('filename').text

            # Extract page number from the filename
            try:
                file_name_root, ext = os.path.splitext(img_name)
                page_number = int(file_name_root.split("_")[2])
            except (IndexError, ValueError):
                print(f"Unable to extract page number from {img_name}. Skipping this file.")
                continue

            if page_number <= min_page_number:
                continue

            img_path = os.path.join(input_image_folder, img_name)
            img = cv2.imread(img_path)

            count = 0

            for obj in root.iter('object'):
                label = obj.find('name').text
                bndbox = obj.find('bndbox')
                xmin = int(bndbox.find('xmin').text)
                ymin = int(bndbox.find('ymin').text)
                xmax = int(bndbox.find('xmax').text)
                ymax = int(bndbox.find('ymax').text)

                cropped_img = img[ymin:ymax, xmin:xmax]

                base_name = os.path.splitext(img_name)[0]

                cropped_img_name = f"{base_name}_{count}.tiff"  # Save as TIFF
                cropped_img_path = os.path.join(output_image_folder, cropped_img_name)
                if os.path.exists(cropped_img_path):
                    pass
                else:
                    cv2.imwrite(os.path.join(cropped_img_path), cropped_img)

                # Save ground truth text file
                gt_text_filename = f"{base_name}_{count}.gt.txt"

                gt_text_filepath = os.path.join(output_label_folder, gt_text_filename)
                if os.path.exists(gt_text_filepath):
                    pass
                else:
                    with open(os.path.join(gt_text_filepath), 'w', encoding='utf-8') as gt_file:
                        gt_file.write(label)

                count += 1
                
                


In [19]:
extract_text_from_image(12)